타이타닉 데이터셋을 가지고 머신러닝이 아닌 딥러닝을 적용해보았습니다.

### 1. Data loading & Preprocessing

**필요한 라이브러리를 Import 합니다.**

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**데이터를 로드합니다.**

In [27]:
titanic_df = pd.read_csv("titanic.csv")
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**데이터를 딥러닝 돌리기 위해 다듬어 줍니다.**

In [28]:
del titanic_df['Cabin'] # 너무 많은 결측치가 존재하기 때문에 제거합니다.
del titanic_df['PassengerId'] # Passenger 번호는 큰 의미를 갖고있지 않은 일련번호를 제거합니다.
del titanic_df['Ticket'] # ticket 번호에서 패턴이 확인되지 않음으로 제거합니다.


titanic_df['Title'] = titanic_df['Name'].str.extract('([A-Za-z]+)\.', expand=False) # 정규식을 사용하여 필요한 단어만 추출합니다.
del titanic_df['Name']

rare_title = [] # 호칭중에 갯수가 10개를 넘지 않는 호칭은 rare_title 빈 리스트 안에 넣어줍니다.
for title in set(titanic_df['Title']):
    if list(titanic_df['Title']).count(title) < 10:
        rare_title.append(title)

# 알 수 없는 호칭들은 알아볼 수 있게 바꿔줍니다.
titanic_df['Title'] = titanic_df['Title'].replace('Mlle', 'Miss') # Mademoiselle
titanic_df['Title'] = titanic_df['Title'].replace('Ms', 'Miss') 
titanic_df['Title'] = titanic_df['Title'].replace('Mme', 'Mrs') # Madame
titanic_df['Title'] = titanic_df['Title'].replace(rare_title, 'Rare')

# 딥러닝에서는 string 값은 읽을 수 없으니 모두 int로 만들어 줍니다.
title_mapping = {"Master":1, "Miss":2, "Mr":3, "Mrs":4, "Rare":5 }
titanic_df['Title'] = titanic_df['Title'].map(title_mapping)
titanic_df['Title'] = titanic_df['Title'].fillna(0)
titanic_df['Title'].astype(int)

# 성별도 int형으로 바꿔줍니다.
sex_mapping = {"male": 0 , "female":1} 
titanic_df['Sex'] = titanic_df['Sex'].map(sex_mapping)

# 승선 열에서 빈 곳을 'S'로 채워줍니다.
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')

# 승선 열도 string형을 int 형으로 모두 mapping 시켜줍니다.
mapping_data ={"S":0, "Q":1, "C":2}
titanic_df["Embarked"] = titanic_df["Embarked"].map(mapping_data) 

# 운임 열도 딥러닝의 성능을 높이기 위해 category를 나눠줍니다.
titanic_df.loc[ titanic_df['Fare'] <= 102, 'Fare'] = 0,
titanic_df.loc[(titanic_df['Fare'] > 102) & (titanic_df['Fare'] <= 204), 'Fare'] = 1,
titanic_df.loc[(titanic_df['Fare'] > 204) & (titanic_df['Fare'] <= 307), 'Fare'] = 2,
titanic_df.loc[ titanic_df['Fare'] > 307, 'Fare'] = 4

# 'Sibsp'와 'Parch'를 'FamilySize'로 합치고 남은 열은 제거해줍니다.
titanic_df["FamilySize"] = titanic_df["SibSp"] + titanic_df["Parch"] +1
del titanic_df['SibSp']
del titanic_df['Parch']

# 혼자인 가구수를 알기 위한 열
titanic_df['isAlone'] = 0
titanic_df.loc[titanic_df['FamilySize'] == 1, 'isAlone'] = 1

# FamilySize 1 ~ 9를 재정렬합니다.
family_mapping = {1: 0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4}
titanic_df['FamilySize'] = titanic_df['FamilySize'].map(family_mapping)

# Age의 N/A를 Age 열의 중앙값으로 채워줍니다.
titanic_df["Age"].fillna(titanic_df.groupby("Title")["Age"].transform("median"), inplace=True)

# Age의 구간을 16씩 잘라서 나눠줍니다.
titanic_df.loc[ titanic_df['Age'] <= 16, 'Age'] = 0,
titanic_df.loc[(titanic_df['Age'] > 16) & (titanic_df['Age'] <= 32), 'Age'] = 1,
titanic_df.loc[(titanic_df['Age'] > 32) & (titanic_df['Age'] <= 48), 'Age'] = 2,
titanic_df.loc[(titanic_df['Age'] > 48) & (titanic_df['Age'] <= 64), 'Age'] = 3,
titanic_df.loc[ titanic_df['Age'] > 64, 'Age'] = 4

# 데이터 전처리 된 데이터프레임을 출력합니다.
titanic_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,isAlone
0,0,3,0,1.0,0.0,0,3,0.4,0
1,1,1,1,2.0,0.0,2,4,0.4,0
2,1,3,1,1.0,0.0,0,2,0.0,1
3,1,1,1,2.0,0.0,0,4,0.4,0
4,0,3,0,2.0,0.0,0,3,0.0,1


### Devide dataframe into X & Y

In [29]:
# 필요한 라이브러리를 임포트 합니다.
from sklearn import model_selection

In [30]:
# 뽑아내고 싶은 데이터 열을 따로 만듭니다.
titanic_target = titanic_df[['Survived']].copy()
titanic_data = titanic_df.copy()
del titanic_data['Survived']

In [31]:
# model_selection을 통해 데이터를 Train Data와 Test Data로 나눠줍니다.
train_data, test_data, train_label, test_label = model_selection.train_test_split(titanic_data, titanic_target, test_size=0.3, random_state=0)

print(train_data.shape)
print(test_data.shape)
print(train_label.shape)
print(test_label.shape)

(623, 8)
(268, 8)
(623, 1)
(268, 1)


In [32]:
test_label.head()
# index를 보면 잘 섞여진 것을 보실 수 있습니다.

,Survived
495,0
648,0
278,0
31,1
255,1


### 3. Change normal labels to one-hot labels

In [33]:
# 원하는 라이브러리를 임포트합니다.
from sklearn.preprocessing import OneHotEncoder

#### Onehotencoder란?
>단 하나의 값만 True이고 나머지는 모두 False인 인코딩을 말한다.
>즉, 1개만 Hot(True)이고 나머지는 Cold(False)이다.
>예를들면 [0, 0, 0, 0, 1]이다. 5번째(Zero-based 인덱스이므로 4)만 1이고 나머지는 0이다.

In [34]:
enc = OneHotEncoder(categories='auto')

In [35]:
# train_label을 OneHotEncoder에 fitting한다.
enc.fit(train_label)

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [36]:
train_label = enc.transform(train_label).toarray()
train_label

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [37]:
# test_label을 OneHotEncoder에 fitting한다.
enc.fit(test_label)

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [38]:
test_label = enc.transform(test_label).toarray()
test_label

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [39]:
print(train_label.shape)
print(test_label.shape)

(623, 2)
(268, 2)


### 4. Build & Train the model

In [40]:
# 딥러닝을 사용하기 위해 tensorflow와 tensorflow의 layers를 임포트 해줍니다.
import tensorflow as tf
from tensorflow import layers

In [41]:
import os
tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [42]:
# X_data의 열의 개수를 맞춰서 작성해야 한다.
X = tf.placeholder(tf.float32, [None, 8])
# Y_data의 열의 개수를 맞춰서 작성해야 한다.(One-Hot Vector)
Y = tf.placeholder(tf.float32, [None, 2])

In [43]:
# layers.dropout()은 True/False로 Training/Testing 여부를 결정해 줄 수 있습니다.
dropout_sign = tf.placeholder(tf.bool)